In [22]:

import sys
import requests
from operator import add

from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

from pyspark.sql.types import *
from pyspark.sql import functions as func
from pyspark.sql.functions import *

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [61]:
import re
import sys
import numpy as np
from pyspark.sql import functions as F
tFile="///content//SmallTestingData (2).txt.bz2"
df0 = spark.read.csv(tFile,header=True)
df0.show(3)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

AnalysisException: ignored

In [70]:
import re
import sys
import numpy as np
from pyspark.sql import functions as F
tFile="/content/SmallTestingData (2).txt.bz2"
df0 = spark.read.csv(tFile,header=True)
df0.show(20)



+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [84]:
# Remove html tags from text
#df0 = df0.withColumn("text_C", F.regexp_replace(F.col("text"), r'<[^>]+>', ""));
#df0.groupBy("doc id").count().show()

# Data 

In [85]:
#Sample the data for faster model training (use the full dataset in reality)
df0 = df0.sample(0.25, seed=200)
# Split the data in train and test (80%-20%)
df, test = df0.randomSplit(weights=[0.8,0.2], seed=200)

In [86]:
#df0.groupBy("doc id").count().show()

AnalysisException: ignored

In [ ]:
test.groupBy("label").count().show()

In [ ]:
# Create a weight of each class
from pyspark.sql import functions as F
p_weight = df.filter('label == 1').count()/ df.count()
n_weight = df.filter('label == 0').count()/ df.count()
print(n_weight, p_weight)

In [32]:
df = df.withColumn("weight", F.when(F.col("label")==1,n_weight).otherwise(p_weight))
df.show(5)

+--------------------+---------+-----+--------------------+-------------------+
|                text|sentiment|label|              text_c|             weight|
+--------------------+---------+-----+--------------------+-------------------+
| Så som i himmele...| positive|    1| Så som i himmele...|0.49901048881852367|
| While sporadical...| negative|    0| While sporadical...| 0.5009895111814764|
|'Anita and Me' is...| negative|    0|'Anita and Me' is...| 0.5009895111814764|
|'Be With Me' is a...| negative|    0|'Be With Me' is a...| 0.5009895111814764|
|'Boogie Nights' u...| positive|    1|'Boogie Nights' u...|0.49901048881852367|
+--------------------+---------+-----+--------------------+-------------------+
only showing top 5 rows



In [93]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import PCA
from pyspark.ml.feature import ChiSqSelector
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import LinearSVC
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import GBTClassifier

from pyspark.ml import Pipeline

# Data transformation

In [94]:
# Preprocessing pipeline
tokenizer = Tokenizer(inputCol="text_c", outputCol="words",)
remover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol="filtered")
# hashingTF = HashingTF(inputCol=remover.getOutputCol(), outputCol="rawFeatures",numFeatues=5000)
countVectorizer = CountVectorizer(inputCol=remover.getOutputCol(), outputCol="rawFeatures", vocabSize=500)
idf = IDF(inputCol=countVectorizer.getOutputCol(), outputCol="featuresIDF")
pipeline_p = Pipeline(stages=[tokenizer,remover, countVectorizer, idf])

In [35]:
# Train the model
data_model = pipeline_p.fit(df)

In [95]:

transformed_data = data_model.transform(df)
transformed_data.show(5)

IllegalArgumentException: ignored

In [37]:
# Transfomr the test data
transformed_test = data_model.transform(test)
transformed_test.show(5)

+--------------------+---------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|sentiment|label|              text_c|               words|            filtered|         rawFeatures|         featuresIDF|
+--------------------+---------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|'Blue Desert' may...| negative|    0|'Blue Desert' may...|['blue, desert', ...|['blue, desert', ...|(500,[0,1,5,10,15...|(500,[0,1,5,10,15...|
|'The Mill on the ...| negative|    0|'The Mill on the ...|['the, mill, on, ...|['the, mill, flos...|(500,[0,1,2,4,7,8...|(500,[0,1,2,4,7,8...|
|(Spoilers) Cash C...| negative|    0|(Spoilers) Cash C...|[(spoilers), cash...|[(spoilers), cash...|(500,[3,11,15,47,...|(500,[3,11,15,47,...|
|(Very light spoil...| negative|    0|(Very light spoil...|[(very, light, sp...|[(very, light, sp...|(500,[1,4,5,9,10,...|(500,[1,4,5,9,

In [38]:
# Print the sages of the pipeline
data_model.stages

[Tokenizer_c49e70add9a5,
 StopWordsRemover_6d49e63c9ba7,
 CountVectorizerModel: uid=CountVectorizer_681512801702, vocabularySize=500,
 IDFModel: uid=IDF_4d1b86245b4d, numDocs=10106, numFeatures=500]

In [39]:
# Get the vocabulary of the CountVectroizer
data_model.stages[2].vocabulary[:20]

['movie',
 'film',
 'one',
 'like',
 'good',
 'even',
 'really',
 'see',
 '-',
 'get',
 'much',
 'story',
 'time',
 'also',
 'great',
 'make',
 'first',
 'people',
 'made',
 'think']

# Metics for the model

In [96]:
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.mllib.evaluation import MultilabelMetrics
from pyspark.mllib.evaluation import MulticlassMetrics

def m_metrics(ml_model,test_data):
    predictions = ml_model.transform(test_data).cache()
    predictionAndLabels = predictions.select("label","prediction").rdd.map(lambda x: (float(x[0]), float(x[1]))).cache()
    
    # Print some predictions vs labels
    print(predictionAndLabels.take(10))
    metrics = MulticlassMetrics(predictionAndLabels)
    
    # Overall statistics
    precision = metrics.precision(1.0)
    recall = metrics.recall(1.0)
    f1Score = metrics.fMeasure(1.0)
    print(f"Precision = {precision:.3f} Recall = {recall:.3f} F1 Score = {f1Score:.3f}")

    # Statistics by class
    labels = [0.0, 1.0]
    for label in sorted(labels):
        print("Class %s precision = %s" % (label, metrics.precision(label)))
        print("Class %s recall = %s" % (label, metrics.recall(label)))
        print("Class %s F1 Measure = %s" % (label, metrics.fMeasure(label, beta=1.0)))

    # Weighted stats
    print("Weighted recall = %s" % metrics.weightedRecall)
    print("Weighted precision = %s" % metrics.weightedPrecision)
    print("Weighted F(1) Score = %s" % metrics.weightedFMeasure())
    print("Weighted F(0.5) Score = %s" % metrics.weightedFMeasure(beta=0.5))
    print("Weighted false positive rate = %s" % metrics.weightedFalsePositiveRate)
    print("Confusion matrix \n", metrics.confusionMatrix().toArray().astype(int))

def m_metrics_l(ml_model,test_data):
    predictions = ml_model.transform(test_data).cache()
    predictionAndLabels = predictions.select("label","prediction").rdd.map(lambda x: (float(x[0]), float(x[1]))).cache()
    
    # Print some predictions vs labels
    # print(predictionAndLabels.take(10))
    metrics = MulticlassMetrics(predictionAndLabels)
    
    # Overall statistics
    precision = metrics.precision(1.0)
    recall = metrics.recall(1.0)
    f1Score = metrics.fMeasure(1.0)
    print(f"Precision = {precision:.4f} Recall = {recall:.4f} F1 Score = {f1Score:.4f}")
    print("Confusion matrix \n", metrics.confusionMatrix().toArray().astype(int))

# ML Model

In [97]:
import time
#selector = ChiSqSelector(numTopFeatures=200, featuresCol=idf.getOutputCol(), outputCol="features", labelCol="label")
cassifier = LogisticRegression(maxIter=5, featuresCol = "featuresIDF", weightCol="weight")
#cassifier = LinearSVC(maxIter=10,  weightCol="weight")
#cassifier = DecisionTreeClassifier(featuresCol=idf.getOutputCol(), weightCol="weight")
start = time.time()
#cassifier = GBTClassifier(maxIter=50, featuresCol = "featuresIDF", weightCol="weight")
pipeline = Pipeline(stages=[cassifier])
print(f"Training started.")
model = pipeline.fit(transformed_data)
print(f"Model created in {time.time()-start:.2f}s.")
m_metrics_l(model,transformed_test)
print(f"Total time {time.time()-start:.2f}s.")

Training started.
Model created in 5.31s.
Precision = 0.8340 Recall = 0.7873 F1 Score = 0.8100
Confusion matrix 
 [[1041  207]
 [ 281 1040]]
Total time 11.11s.


In [42]:
cassifier = GBTClassifier(maxIter=10, featuresCol = "featuresIDF", maxDepth=10)
pipeline = Pipeline(stages=[cassifier])
start = time.time()
print(f"Training started.")
model = pipeline.fit(transformed_data)
print(f"Model created in {time.time()-start:.2f}s.")
m_metrics_l(model,transformed_test)
print(f"Total time {time.time()-start:.2f}s.")

Training started.
Model created in 38.94s.
Precision = 0.8268 Recall = 0.6962 F1 Score = 0.7559
Confusion matrix 
 [[ 872  216]
 [ 450 1031]]
Total time 43.26s.


In [43]:
cassifier = LinearSVC(maxIter=10, featuresCol = "featuresIDF", weightCol="weight")
pipeline = Pipeline(stages=[cassifier])
start = time.time()
print(f"Training started.")
model = pipeline.fit(transformed_data)
print(f"Model created in {time.time()-start:.2f}s.")
m_metrics_l(model,transformed_test)
print(f"Total time {time.time()-start:.2f}s.")

Training started.
Model created in 4.90s.
Precision = 0.8244 Recall = 0.7829 F1 Score = 0.8031
Confusion matrix 
 [[1037  219]
 [ 285 1028]]
Total time 9.16s.


In [44]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
layers = [500, 100, 20, 2]

# create the trainer and set its parameters
cassifier = MultilayerPerceptronClassifier(maxIter=10, layers=layers,featuresCol = "featuresIDF", blockSize=128, seed=1234)
pipeline = Pipeline(stages=[cassifier])
start = time.time()
print(f"Training started.")
model = pipeline.fit(transformed_data)
print(f"Model created in {time.time()-start:.2f}s.")
m_metrics_l(model,transformed_test)
print(f"Total time {time.time()-start:.2f}s.")

Training started.
Model created in 8.30s.
Precision = 0.8332 Recall = 0.7736 F1 Score = 0.8023
Confusion matrix 
 [[1018  208]
 [ 304 1039]]
Total time 12.06s.
